In [ ]:
#imports
import pandas as pd 
import numpy as np

In [ ]:
raw_ds=pd.read_csv(r"..\data\raw\dummy.csv")

In [ ]:
raw_ds

In [ ]:
#consolidate data only collected on initial visit and given -4 later on
#find cols, rows where -4 exists
reduced= raw_ds.loc[raw_ds['NACCVNUM']!=1]
nan_bool=reduced.where(reduced.eq(-4)).stack().index.values #fastest;
nan_ind = np.unique([ seq[0] for seq in nan_bool ])
#find 1st visit values  of those columns. do all patients have a first visit? sanity chk
first_visit=raw_ds.loc[raw_ds['NACCVNUM']==1]
print(raw_ds.NACCID.nunique()==first_visit.NACCID.nunique())
#replace per group
con_raw_ds=raw_ds.copy()
for n in raw_ds.NACCID.unique():
    temp=raw_ds.loc[raw_ds.NACCID==n]#for each NACCID find its rows
    nan_match=np.intersect1d(nan_ind,temp.index) #if it has row/cols in nan_ind
    if len(nan_match)>0:
        temp2=first_visit.loc[first_visit.NACCID==n]
        for nm in nan_match:
            cols=[x[1] for x in nan_bool if x[0] == nm] 
            #reduce the col number
            red_cols=temp2[cols].columns[(temp2[cols] !=-4).iloc[0]]
            #replace by corresponding values in first visit
            con_raw_ds.loc[nm,red_cols]=temp2[red_cols].values[0]